In [ ]:
import tensorflow as tf
import numpy as np
import os

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
#   """Model function for CNN."""
  
    # Input Layer
    # input layer shape should be [batch_size, image_width, image_height, channels] for conv2d
    # set batch_size = -1 means batch_size = the number of input
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1 and Pooling Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],# or kernel_size=5
        padding="same", #same; output tensor should have the same width and height as input tensor; otherwise: valid(default)
        activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1,
        pool_size=[2, 2],
        strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(
        inputs=pool2_flat,
        units=1024, # number of neurons in the dense layer
        activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.4,
        training= mode==tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")
    
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
          tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100, # number of data in a minibatch
        num_epochs=None,
        shuffle=True) # shuffle training data
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11dc2f470>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow

INFO:tensorflow:loss = 2.30468, step = 2
INFO:tensorflow:probabilities = [[ 0.11481094  0.09720138  0.10500486  0.12018012  0.08550931  0.08602919
   0.08281601  0.11138163  0.10376552  0.09330107]
 [ 0.13933764  0.08510371  0.12095428  0.11031881  0.08520638  0.09016699
   0.0967098   0.0965067   0.09749559  0.07820014]
 [ 0.12610717  0.07977467  0.1115299   0.11905928  0.10395599  0.07547995
   0.11071729  0.08445352  0.09193301  0.09698924]
 [ 0.10226682  0.08807037  0.10342886  0.09496982  0.11207826  0.08513803
   0.08962141  0.10015685  0.11088292  0.11338665]
 [ 0.10560487  0.11596462  0.10307478  0.10114637  0.09060384  0.09922973
   0.09163126  0.09104952  0.10378022  0.09791478]
 [ 0.11164349  0.08735465  0.10883458  0.1186043   0.10684211  0.08470545
   0.08727153  0.10777659  0.09652258  0.0904448 ]
 [ 0.10686507  0.10347467  0.10752127  0.09551563  0.08448651  0.09662532
   0.11362588  0.09940616  0.0983833   0.0940962 ]
 [ 0.10072142  0.08922431  0.10385881  0.11922912  0

INFO:tensorflow:global_step/sec: 5.65017
INFO:tensorflow:probabilities = [[ 0.11835428  0.09757586  0.09085041  0.07829145  0.12857679  0.09304211
   0.11311168  0.0633909   0.12035578  0.09645069]
 [ 0.1048095   0.06165748  0.0716417   0.07481039  0.09701896  0.06408802
   0.12961876  0.0656229   0.13045277  0.2002795 ]
 [ 0.06321879  0.08314568  0.05523679  0.11611728  0.07626517  0.11152899
   0.08008415  0.1085045   0.16107903  0.14481962]
 [ 0.08251372  0.09136397  0.06642924  0.11441489  0.06067318  0.09412236
   0.10995404  0.09539153  0.14883433  0.13630269]
 [ 0.05970869  0.09282526  0.06418713  0.11649729  0.09898575  0.07300503
   0.07116633  0.15144178  0.10591253  0.16627029]
 [ 0.10289177  0.06227181  0.08564132  0.10372366  0.10756701  0.05401699
   0.1094127   0.10410379  0.12938671  0.14098433]
 [ 0.07686883  0.11171914  0.08971421  0.10086814  0.11161415  0.08157033
   0.09863884  0.0933233   0.13812517  0.09755793]
 [ 0.15702778  0.02237162  0.14534386  0.09423945  0

INFO:tensorflow:loss = 1.95983, step = 102 (17.702 sec)
INFO:tensorflow:probabilities = [[ 0.01201061  0.02286766  0.05770111  0.0290229   0.09440994  0.01071209
   0.64767998  0.01463768  0.05546834  0.05548963]
 [ 0.02294661  0.5060097   0.02923485  0.04494118  0.04797889  0.02633674
   0.03422107  0.08288858  0.1194497   0.08599257]
 [ 0.09353439  0.00208683  0.77838743  0.00955374  0.00733416  0.02239157
   0.05753693  0.00354585  0.02476615  0.00086302]
 [ 0.01820268  0.51104021  0.04057097  0.07845809  0.02901618  0.03631439
   0.0366003   0.07001979  0.11155782  0.06821956]
 [ 0.01317815  0.1368532   0.02372291  0.03305383  0.16486457  0.0497327
   0.0647072   0.09665831  0.3048172   0.112412  ]
 [ 0.04312094  0.0428693   0.02699007  0.13490388  0.04410278  0.08709301
   0.20297997  0.09922635  0.1447227   0.1739911 ]
 [ 0.87338066  0.00096592  0.01817539  0.0095156   0.01496621  0.02115017
   0.00623413  0.00337248  0.04882195  0.00341749]
 [ 0.08715538  0.06434572  0.03854291 

INFO:tensorflow:global_step/sec: 5.99949
INFO:tensorflow:probabilities = [[ 0.0082191   0.00069749  0.92594582  0.0256211   0.00135488  0.00482405
   0.01229313  0.00107558  0.01965266  0.00031614]
 [ 0.52540928  0.00075977  0.02172598  0.00830919  0.18917669  0.0380419
   0.11059338  0.01703979  0.04417164  0.04477239]
 [ 0.21406834  0.00099145  0.02085948  0.2078858   0.0018926   0.50544792
   0.02578379  0.00132399  0.0201085   0.00163815]
 [ 0.01083629  0.03561589  0.00466118  0.0248023   0.02227173  0.39446628
   0.01584908  0.02097493  0.29396507  0.17655727]
 [ 0.00969443  0.04472364  0.50019962  0.03575075  0.01225811  0.01806094
   0.28165656  0.00356975  0.08380676  0.01027943]
 [ 0.94402629  0.00001216  0.01568548  0.01064505  0.00029752  0.0146377
   0.00220068  0.00579038  0.00549777  0.00120692]
 [ 0.8641547   0.00005154  0.0404212   0.06652204  0.00016163  0.01745606
   0.00098518  0.00139032  0.00855609  0.00030123]
 [ 0.00094622  0.37137309  0.0096035   0.01717918  0.0

...

INFO:tensorflow:loss = 0.0639973, step = 9202 (17.104 sec)
INFO:tensorflow:probabilities = [[ 0.00000003  0.00000319  0.9995172   0.0000001   0.          0.          0.
   0.00047931  0.00000027  0.        ]
 [ 0.00002245  0.00000078  0.9995684   0.00018147  0.00019774  0.00000371
   0.0000009   0.00001831  0.00000031  0.00000586]
 [ 0.0000016   0.00003138  0.0000065   0.00000025  0.99976259  0.000002
   0.00003363  0.00001283  0.00003894  0.00011031]
 [ 0.00008223  0.00000017  0.00000521  0.          0.99974304  0.
   0.00016102  0.00000004  0.00000803  0.00000014]
 [ 0.00000001  0.00000001  0.          0.          0.00000006  0.          0.
   0.9999994   0.          0.00000051]
 [ 0.00000667  0.99966633  0.00000734  0.00000109  0.00017581  0.00000007
   0.00000213  0.00010776  0.00002922  0.00000356]
 [ 0.00000018  0.00000015  0.00007284  0.00004706  0.00000011  0.00000151
   0.00000115  0.00000126  0.99987471  0.00000093]
 [ 0.          0.          0.00000061  0.99994218  0.       

INFO:tensorflow:global_step/sec: 5.85281
INFO:tensorflow:probabilities = [[ 0.00000463  0.00000744  0.99871552  0.00060608  0.00004978  0.00000415
   0.00000002  0.00009342  0.00022009  0.00029893]
 [ 0.00000823  0.00000017  0.00000138  0.00000003  0.00000499  0.00002085
   0.99996376  0.00000002  0.00000032  0.00000017]
 [ 0.00000154  0.99989235  0.00001289  0.00000048  0.00000544  0.00000082
   0.00003004  0.00001688  0.00002978  0.00000969]
 [ 0.00001482  0.0000048   0.000004    0.00002516  0.00000087  0.99982953
   0.00003139  0.00000096  0.00008804  0.00000044]
 [ 0.          0.00000001  0.00000014  0.9999038   0.00000001  0.00002143
   0.          0.00000001  0.00007166  0.00000293]
 [ 0.00000492  0.          0.00000006  0.0000003   0.00001071  0.00000005
   0.00000004  0.00000176  0.00003863  0.99994349]
 [ 0.0000015   0.00000204  0.00000069  0.00007718  0.00013181  0.00001453
   0.0000001   0.00001175  0.99940562  0.00035485]
 [ 0.00000015  0.00000002  0.00000012  0.00000001  0

INFO:tensorflow:loss = 0.0254735, step = 9302 (17.085 sec)
INFO:tensorflow:Saving checkpoints for 9319 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00000133  0.99983883  0.00011154  0.00000023  0.00000052  0.
   0.00000001  0.00003074  0.00001661  0.0000001 ]
 [ 0.00000645  0.00005249  0.99916685  0.00032398  0.00000007  0.00000039
   0.00000003  0.00039359  0.0000553   0.00000096]
 [ 0.          0.          0.          0.          0.          0.99999928
   0.          0.          0.00000068  0.00000002]
 [ 0.00001966  0.00000384  0.00000148  0.00067331  0.00000559  0.99788672
   0.00000125  0.00057765  0.00022308  0.00060743]
 [ 0.          0.00000053  0.99999571  0.00000333  0.          0.          0.
   0.          0.00000046  0.        ]
 [ 0.00000599  0.00000015  0.00000212  0.          0.00001378  0.00000277
   0.99997401  0.          0.00000109  0.        ]
 [ 0.          0.00000063  0.0000001   0.00000007  0.00044869  0.00000032
   0.00004852  

INFO:tensorflow:global_step/sec: 5.48222
INFO:tensorflow:probabilities = [[ 0.00005344  0.00001145  0.00000825  0.00000025  0.99928087  0.00000002
   0.00001613  0.00000157  0.00000019  0.00062788]
 [ 0.00000005  0.00000026  0.00000017  0.00012961  0.00064752  0.00001125
   0.          0.00117349  0.00001656  0.99802113]
 [ 0.          0.00107117  0.99890482  0.00000572  0.00000212  0.
   0.00000001  0.00000365  0.00001251  0.00000002]
 [ 0.          0.00000003  0.00000001  0.99990296  0.          0.00004984
   0.          0.00000001  0.00000143  0.00004582]
 [ 0.00000015  0.00000048  0.00000005  0.00000032  0.99608165  0.00000307
   0.00024389  0.00000843  0.00046721  0.00319475]
 [ 0.00000892  0.          0.          0.00000003  0.00000022  0.00010008
   0.99989021  0.          0.0000005   0.        ]
 [ 0.00000477  0.09238795  0.00013747  0.00023563  0.00047637  0.0134757
   0.88964826  0.00000027  0.00363248  0.00000119]
 [ 0.9999994   0.00000003  0.00000056  0.          0.        

INFO:tensorflow:loss = 0.0189984, step = 9402 (18.242 sec)
